<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Projetos de Análise de Dados com Linguagem Python</font>
# <font color='blue'>Projeto 1 - Extraindo e Analisando Dados de Bancos de Dados</font>

## Pacotes Python Usados no Projeto

In [1]:
# Instala o pacote watermark #marca da agua
!pip install -q watermark

In [ ]:
pip install --upgrade pip

In [ ]:
# Instala o pacote iPython-sql #para usar linguagem SQL
!pip install ipython-sql

In [ ]:
pip install jupysql

In [ ]:
pip install --upgrade ipython-sql prettytable

In [6]:
import sqlite3
print(sqlite3.sqlite_version)

3.45.1


In [9]:
# Imports
import os #acessar o sistema operacional
import pandas as pd
import sqlite3 #banco de dados

https://www.sqlite.org/about.html

In [10]:
%reload_ext watermark
%watermark -a "DSA: Murilo Farias"

Author: DSA: Murilo Farias



## Criando o Banco de Dados Relacional

In [11]:
# Cria um dataframe com os dados de origem
dados = pd.DataFrame({'nivel': ['Junior', 'Pleno', 'Senior'],
                      'salario': [7500, 14650, 18320],
                      'cargo': ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']})

In [12]:
# Exibe os dados
dados.head()

,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [22]:
# Define o caminho para o arquivo do banco de dados
arquivo_path = 'db_dsa_mf.db'

In [23]:
# Verifica se o arquivo existe e deleta se existir para criar um novo arquivo depois
if os.path.exists(arquivo_path):
    try:
        os.remove(arquivo_path)
        print(f"Arquivo {arquivo_path} deletado com sucesso!")
    except Exception as e:
        print(f"Erro ao deletar o arquivo {arquivo_path}. Detalhes: {e}")
else:
    print(f"Arquivo {arquivo_path} não encontrado.")

Arquivo db_dsa_mf.db não encontrado.


## Conectando ao Banco de Dados com Linguagem Python

In [24]:
# Criamos e conectamos a conexão a um banco de dados SQLite
cnn = sqlite3.connect('db_dsa_mf.db')

In [25]:
# Copia o dataframe para dentro do banco de dados como uma tabela
#dado criado no dicionario
#to_sql funcao
#criar tabela funcionario
#no banco cnn
dados.to_sql('funcionarios', cnn)

3

In [26]:
# Carregamos a extensão SQL
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [27]:
%reload_ext sql

In [28]:
# Definimos o banco de dados
%sql sqlite:///db_dsa_mf.db

Connecting and switching to connection 'sqlite:///db_dsa_mf.db'

> Agora executamos nossas consultas SQL usando diretamente Linguagem SQL dentro do Jupyter Notebook.

In [31]:
%%sql

SELECT * FROM funcionarios

Running query in 'sqlite:///db_dsa_mf.db'

index,nivel,salario,cargo
0,Junior,7500,Analista de Dados
1,Pleno,14650,Cientista de Dados
2,Senior,18320,Engenheiro de Dados


In [ ]:
import os
import sqlite3

# Caminho relativo do banco de dados
arquivo_path = 'db_mf.db'

# Caminho absoluto
caminho_absoluto = os.path.abspath(arquivo_path)
print(f'Caminho absoluto: {caminho_absoluto}')


In [ ]:
%%sql

SELECT count(*) FROM funcionarios

In [ ]:
%%sql

SELECT round(avg(salario),2) as 'salario_medio' FROM funcionarios

## Carregando o Banco de Dados a Partir de Arquivos CSV

Temos em mãos um arquivo com dados de pacientes que desenvolveram ou não diabetes. Vamos colocar o conteúdo do arquivo em um banco de dados.

In [ ]:
# Carrega o dataset
df = pd.read_csv('dataset.csv')

In [ ]:
type(df)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Copia o dataframe para dentro do banco de dados como uma tabela
df.to_sql('diabetes', cnn)

In [ ]:
%%sql

SELECT count(*) FROM diabetes

## Sintaxe SQL e Sintaxe Pandas

> Retorne os pacientes que têm BMI maior que 52 e idade (Age) entre 25 e 30 anos.

**Sintaxe de consulta do Pandas:**

In [ ]:
df.query("BMI > 52 and 25 <= Age <= 30")

**Sintaxe de consulta com SQL:**

In [ ]:
%%sql

SELECT * FROM diabetes WHERE BMI > 52 AND Age BETWEEN 25 AND 30

## Respondendo Perguntas de Negócio com Análise de Dados

Precisamos gerar uma amostra de dados com os pacientes com mais de 50 anos e para cada um deles indicar em uma nova coluna se o paciente está normal (BMI menor que 30) ou obeso (BMI maior ou igual a 30). Então devemos gerar um novo arquivo CSV e encaminhar para o tomador de decisão.

**Retorne Age, Glucose e Outcome para pacientes com Glucose maior que 195.**

In [ ]:
%%sql

SELECT Age, Glucose, Outcome FROM diabetes WHERE Glucose > 195

In [ ]:
df.columns

**Vamos criar uma nova tabela no banco de dados.**

In [ ]:
%%sql

CREATE TABLE pacientes (Pregnancies INT,
                        Glucose INT,
                        BloodPressure INT,
                        SkinThickness INT,
                        Insulin INT,
                        BMI DECIMAL(8, 2),
                        DiabetesPedigreeFunction DECIMAL(8, 2),
                        Age INT,
                        Outcome INT);

**A tabela está vazia.**

In [ ]:
%%sql

SELECT * FROM pacientes

**Vamos copiar o conteúdo de uma tabela para outra, mas somente para pacientes maiores que 50 anos de idade.**

In [ ]:
%%sql

INSERT INTO pacientes(Pregnancies, 
                      Glucose, 
                      BloodPressure, 
                      SkinThickness, 
                      Insulin, 
                      BMI, 
                      DiabetesPedigreeFunction, 
                      Age, 
                      Outcome) 
SELECT Pregnancies, Glucose, BloodPressure, SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome 
FROM diabetes WHERE Age > 50;

**Retorna todos os pacientes.**

In [ ]:
%%sql

SELECT * FROM pacientes

**Vamos alterar a tabela e incluir uma nova coluna.**

In [ ]:
%%sql 

ALTER TABLE pacientes
ADD Perfil VARCHAR(10);

**Coluna criada:**

In [ ]:
%%sql

SELECT * FROM pacientes

**Agora vamos atualizar a coluna com o valor "Normal" se o BMI for menor do que 30.**

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Normal'
WHERE BMI < 30;

**Conferimos o resultado:**

In [ ]:
%%sql

SELECT * FROM pacientes

**Agora vamos atualizar a coluna com o valor "Obeso" se o BMI for maior ou igual a 30.**

In [ ]:
%%sql

UPDATE pacientes
SET Perfil = 'Obeso'
WHERE BMI >= 30;

**Conferimos o resultado:**

In [ ]:
%%sql

SELECT * FROM pacientes

## Retornando os Dados Para o Pandas e Salvando o CSV

In [ ]:
# Query
dsa_query = cnn.execute("SELECT * FROM pacientes")

In [ ]:
dsa_query

In [ ]:
# List Comprehension para retornar os metadados da tabela (nomes de colunas)
cols = [coluna[0] for coluna in dsa_query.description]

In [ ]:
cols

In [ ]:
# Gera o dataframe
resultado = pd.DataFrame.from_records(data = dsa_query.fetchall(), columns = cols)

In [ ]:
# Shape
resultado.shape

In [ ]:
# Visualiza
resultado.head()

In [ ]:
# Salva em CSV
resultado.to_csv('resultado.csv', index = False)

In [ ]:
%reload_ext watermark
%watermark -a "Data Science Academy"

In [ ]:
#%watermark

In [ ]:
#%watermark --iversions

# Fim